In [6]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus']=False

# sql 함수 사용
from pyspark.sql.functions import when, udf, col, regexp_replace
# sql 타입변경
from pyspark.sql.types import DoubleType,IntegerType, StringType
import pyspark.sql.functions as F

In [2]:
sc = SparkContext('local') # 'spark://ip:7070'
sqlCtx = SQLContext( sc )

In [3]:
df = sqlCtx.read.csv(r'c:\data\births.txt',inferSchema=True)
df

DataFrame[_c0: int, _c1: double, _c2: double]

In [4]:
# 컬럼명 변경
df = df.withColumnRenamed('_c0','year').\
        withColumnRenamed('_c1','boys').\
        withColumnRenamed('_c2','girls')

### 집계 함수 사용
- agg(..) 사용
- https://rfriend.tistory.com/213

In [10]:
# 통계, 집합 함수 사용
print(df.agg({'girls':'sum'}).collect()) # 리스트형식
print(df.agg({'girls':'sum'}).collect()[0]['sum(girls)'])

[Row(sum(girls)=164392696.0)]
164392696.0


In [15]:
print(df.agg({'girls':'sum'}).toPandas())
print('-'*20)
print(df.agg({'girls':'sum'}).toPandas().loc[0]['sum(girls)'])

    sum(girls)
0  164392696.0
--------------------
164392696.0


In [17]:
df. agg(F.sum('girls')).show()

+------------+
|  sum(girls)|
+------------+
|1.64392696E8|
+------------+



In [21]:
# bug..
df.agg({'girls':'sum','boys':'avg'}).show()

+------------------+------------+
|         avg(boys)|  sum(girls)|
+------------------+------------+
|1225776.0681818181|1.64392696E8|
+------------------+------------+



In [19]:
df.agg(F.sum('girls'),F.mean('girls')).show()

+------------+------------------+
|  sum(girls)|        avg(girls)|
+------------+------------------+
|1.64392696E8|1245399.2121212122|
+------------+------------------+



In [20]:
df.agg({'girls':'stddev','boys':'mean'}).show()

+------------------+-----------------+
|         avg(boys)|    stddev(girls)|
+------------------+-----------------+
|1225776.0681818181|740002.8662878576|
+------------------+-----------------+



In [22]:
df.agg(F.sum('girls').alias('여아총합'),F.mean('girls').alias('여아평균')).show()

+------------+------------------+
|    여아총합|          여아평균|
+------------+------------------+
|1.64392696E8|1245399.2121212122|
+------------+------------------+

